In [1]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py

--2024-06-20 19:11:13--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-06-20 19:11:13 (9.18 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [13]:
! wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json

--2024-06-27 16:23:43--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json’

documents.json      100%[===================>] 642.90K  --.-KB/s    in 0.04s   

2024-06-27 16:23:43 (17.9 MB/s) - ‘documents.json’ saved [658332/658332]



In [1]:
import minsearch
import json
from openai import OpenAI
import os
from dotenv import load_dotenv

from elasticsearch import Elasticsearch
#start elastic search from docker image first
es_client = Elasticsearch('http://localhost:9200')
es_client.info()

ObjectApiResponse({'name': '853b7f686046', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Mk1wWHTMRgmGbAUhXVIfEQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [2]:
# Load environment variables from the .env file
load_dotenv()

True

In [3]:
os.environ

environ{'XDG_SESSION_ID': '2',
        'TERM_PROGRAM': 'vscode',
        'HOSTNAME': 'ip-172-31-0-159.dev-sunrun.local',
        'SSL_CERT_FILE': '/etc/pki/ca-trust/extracted/pem/tls-ca-bundle.pem',
        'TERM': 'xterm-color',
        'SHELL': '/bin/bash',
        'HISTSIZE': '1000',
        'SSH_CLIENT': '172.31.0.217 55346 22',
        'TERM_PROGRAM_VERSION': '1.89.1',
        'USER': 'ec2-user',
        'LS_COLORS': 'rs=0:di=38;5;27:ln=38;5;51:mh=44;38;5;15:pi=40;38;5;11:so=38;5;13:do=38;5;5:bd=48;5;232;38;5;11:cd=48;5;232;38;5;3:or=48;5;232;38;5;9:mi=05;48;5;232;38;5;15:su=48;5;196;38;5;15:sg=48;5;11;38;5;16:ca=48;5;196;38;5;226:tw=48;5;10;38;5;16:ow=48;5;10;38;5;21:st=48;5;21;38;5;15:ex=38;5;34:*.tar=38;5;9:*.tgz=38;5;9:*.arc=38;5;9:*.arj=38;5;9:*.taz=38;5;9:*.lha=38;5;9:*.lz4=38;5;9:*.lzh=38;5;9:*.lzma=38;5;9:*.tlz=38;5;9:*.txz=38;5;9:*.tzo=38;5;9:*.t7z=38;5;9:*.zip=38;5;9:*.z=38;5;9:*.Z=38;5;9:*.dz=38;5;9:*.gz=38;5;9:*.lrz=38;5;9:*.lz=38;5;9:*.lzo=38;5;9:*.xz=38;5;9:*.bz2=38;

In [4]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
from huggingface_hub import login

In [8]:
login(token= os.environ.get('HG_TOKEN'))

In [9]:
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1", device_map="auto", load_in_4bit=True
)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [14]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")
model_inputs = tokenizer(["Hi what a beatifull day here in Denver area"], return_tensors="pt").to("cuda")
generated_ids = model.generate(**model_inputs)
tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'Hi what a beatifull day here in Denver area. I am a new member and I'

In [16]:
with open('documents.json', 'r') as f:
    doc_raws = json.load(f)
documents = []
for course_dict in doc_raws:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [18]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}
index_name = "module_02_index"
#es_client.indices.create(index = index_name, body = index_settings)
from tqdm.auto import tqdm
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [19]:
index = minsearch.Index(text_fields = ['question', 'text', 'section'],
                         keyword_fields = ['course']
                        )
index.fit(documents)

In [27]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}
    result  = index.search(query = query,
                 boost_dict = boost,
                 num_results = 5
                )
    return result

def elastic_search (query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    es_result = es_client.search(index = index_name, body=search_query)
    es_result_docs = []
    for hit in es_result['hits']['hits']:
        es_result_docs.append(hit['_source'])


    return es_result_docs
    
def build_prompt(query, search_result):
    

    context = ""
    for doc in search_result:
        context =context + f"section :{doc['section']} \nquestion:{doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt_template = """
    You are a course teaching assistant. Answer the given QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    
    QUESTION: {question}\n
    CONTEXT: {context}
    """.strip()
    prompt = prompt_template.format(question = query, 
                                    context = context).strip()
    return prompt

def llm (prompt):
    model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
    generated_ids = model.generate(**model_inputs, max_length = 1000)
    result = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    return result

In [28]:
def rag(query):
    results = elastic_search(query)#search(query)
    prompt = build_prompt(query, results)
    answer = llm(prompt)
    return answer

In [29]:
query = "When will module 3 on llm and rag will start?"
rag(query)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


'You are a course teaching assistant. Answer the given QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n    \n    QUESTION: When will module 3 on llm and rag will start?\n\n    CONTEXT: section :General course-related questions \nquestion:Course - When will the course start?\nanswer:The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours\'\' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club\'s Slack and join the channel.\n\nsection :General course-related questions \nquestion:Course - When will the course start?\nanswer:The purpose of this document is to capture frequently asked technical qu

In [30]:
print(_)